# Implementation AVL and RED-BLACK TREE


*   Store 1000 frequent words from the given file in AVL tree
*   Search same 100 random words and count comparisons
*   Prove the statement, AVL trees offer faster search due to their strict height balancing whereas Red-Black trees are faster while insert and delete operations.





In [1]:
import string
import sys

# ==========================================
# 1. AVL Tree Implementation
# ==========================================

class AVLTree(object):

  class __Node(object):                           # A node in an AVL Tree
    def __init__(self, key, data):                # Contructing a key-data pair since every node must have an item
      self.key = key
      self.data = key, data                       # store item key & data
      self.left = None
      self.right = None
      self.updateHeight()                         # Set initial height of node

    def updateHeight(self):                       # update height of node from childer
      self.height = max(                          # Get maximum child height
          child.height if child is not None else 0
          for child in (self.left, self.right)
      ) + 1

    def heightDiff(self):                         # Return difference in child heights
      left = self.left.height if self.left is not None else 0
      right = self.right.height if self.right is not None else 0
      return left - right                         # Return difference in child heights

  def __init__(self):
    self.__root = None                            # Hold the tree root
    # --- Introducing Counters ---
    self.avl_rotation_count = 0 # Count single rotation
    self.avl_search_comparison = 0 # Count key comparisons during search

  def insert(self, key, data):
    self.__root, flag = self.__insert(self.__root, key, data)

  def print_structure(self):
    self.__print_structure(self.__root,0)

  def search(self, key):                          # Public method to search for a key
    return self.__search(self.__root, key)     # Returns tuple: (found_boolean, comparison_count)

  def __search(self, node, key):
    if node is None:
      return False                       # Key not found

    # --- 2 Node Search Comparison ---
    self.avl_search_comparison +=1 # Increment comparison count for each node visited if key match
    if key == node.key:
      return True                        # Key found

    self.avl_search_comparison +=1
    if key < node.key:
      return self.__search(node.left, key)
    else:
      return self.__search(node.right, key)

    # --- End 2 Node Search Comparison ---

    # # --- 1 Node Search Comparison ---
    # self.avl_search_comparison += 1 # Increment comparison count for each node visited

    # if key == node.key:
    #   return True, count                          # Key found
    # elif key < node.key:
    #   return self.__search(node.left, key, count)
    # else:
    #   return self.__search(node.right, key, count)
    # # --- End 1 Node Search Comparison ---


  def __insert(self, node, key, data):            # Insert an item into an AVL subtree
    if node is None:                              # For an empty subtree, return a new node in a tree
      return AVLTree.__Node(key, data), True

    if key == node.key:                           # If node already has the insert key,
      node.data = data
      return node, False                          # Return the node and False for flag

    elif key < node.key:                          # Check left subtree
      node.left, flag  = self.__insert(node.left, key, data) # If so, insert on left and update the left link
      if node.heightDiff() > 1:                   # if insert made node left heavy
        if key > node.left.key:                   # If key is greater than left child's key (LR case)
          node.left = self.rotateLeft(node.left) # Perform Left rotation on the left child
        node = self.rotateRight(node)           # Then perform Right rotation on the current node

    else:                                         # otherwise key belongs in right subtree
      node.right, flag = self.__insert(node.right, key, data) # Insert it on right and update the link
      if node.heightDiff() < -1:                  # if insert made node right heavy
        if key < node.right.key:                  # If key is less than right child's key (RL case)
          node.right = self.rotateRight(node.right) # Perform Right rotation on the right child
        node = self.rotateLeft(node)            # Then perform Left rotation on the current node

    node.updateHeight()
    return node, flag                           # Return the updated node & insert flag

  def rotateRight(self, top):                   # Rotate a subtree to right
    self.avl_rotation_count += 1
    toRaise = top.left                          # The node to raise is top's left child
    top.left = toRaise.right                    # the raised node's right crosss over
    toRaise.right = top                         # to be the left subtree under the old top
    top.updateHeight()                          # heights must be updated
    toRaise.updateHeight()
    return toRaise                              # Return raised node to update parent

  def rotateLeft(self,top):                     # Roate a subtree to the left
    self.avl_rotation_count += 1
    toRaise = top.right                         # the node to raise is top's right child
    top.right = toRaise.left                    # the raised node's right crosss over
    toRaise.left = top                          # to be the right subtree under the old top
    top.updateHeight()                          # update heights
    toRaise.updateHeight()
    return toRaise                              # Return raised node to update parent

  def delete(self, goal):                       # Delete a nodw whose key matches goal
    self.__root, flag = self.__delete(self.__root, goal) # Delete starting at root and update the root link
    return flag                                 # Return flag indicating goal node found

  def __delete(self, node, goal):               # Delete match goal key from subtree rooted at node
    if node is None:                            # If substree is empty
      return None, False

    flag = False
    # --- 2 Node Search Comparison ---
    self.avl_search_comparison += 1

    if goal == node.key:
      # Node found (Handle deletion below)
      flag = True

    else:
      # Decide direction
      self.avl_search_comparison += 1

      if goal < node.key:                          # Check left subtree
        node.left, flag = self.__delete(node.left, goal) # If so, delete from left
        node = self.__balanceLeft(node)           # Correct any imbalance
      else:                                        # Check right subtree
        node.right, flag = self.__delete(node.right, goal) # If so, delete from right
        node = self.__balanceRight(node)          # Correct any imbalance

    # --- Deletion handling (Key Found or Reached) ---
    if goal == node.key:
      # handle cases where was found at this node:
      if node.left is None:                     # Cases 1: 0 or 1 child (right chid)
        return node.right, flag
      elif node.right is None:                  # Case 2: 1 child (left child)
        return node.left, flag
      # delete node has two children: find successor
      else:
        node.key, node.data, node.right = self.__deleteMin(node.right) # Comparison count in __deleteMin()
        node = self.__balanceRight(node)
        flag = True

    node.updateHeight()                        # Update the height of node after deletion
    return node, flag
    # --- End 2 Node Search Comparison ---

    # # --- 1 Node Search Comparison ---
    # self.avl_search_comparison += 1
    # if goal < node.key:                         # Check left subtree
    #   node.left, flag = self.__delete(node.left, goal) # If so, delete from left
    #   node = self.__balanceLeft(node)           # Correct any imbalance

    # elif goal > node.key:                       # Check right subtree
    #   node.right, flag = self.__delete(node.right, goal) # If so, delete from right
    #   node = self.__balanceRight(node)          # Correct any imbalance

    # elif node.left is None:                     # If no left child
    #   return node.right, True                   # Return right child as remainder, flagging deletion
    # elif node.right is None:                    # If no right child
    #   return node.left, True                    # Return left child as remainder, flagging deletion
    # # Deleted node has two children so find successor in right subtree and replace this item
    # else:
    #   node.key, node.data, node.right = self.__deleteMin (node.right)
    #   node = self.__balanceRight(node)          # Correct any imbalance
    #   flag = True                               # The goal is found and deleted

    # node.updateHeight()                         # Update the height of node after deletion
    # return node, flag

    # # --- End 1 Node search comparison ---

  def __deleteMin(self, node):
    # Comparison 1: Check if left child link is empty
    self.avl_search_comparison += 1

    if node.left is None:                       # If left child link is empty
      return (node.key, node.data, node.right)  # this node is minium and its right subtree, if any, replaces it

    # --- 2 Node Comparison ---
    self.avl_search_comparison += 1             # Remove this for 1 Node Comparison
    # --- End 2 Node Comparison ---

    key, data, node.left = self.__deleteMin(node.left) # Else, delete minimum from left subtree
    node = self.__balanceLeft(node)             # Correct any imbalance
    node.updateHeight()                         # Update the height of node after deletion
    return (key, data, node)

  def __balanceLeft(self, node):                # Rebalance after deletion in left subtree. Node might become right heavy.
    if node.heightDiff() < -1:                  # if node is right heavy
      if node.right.heightDiff() > 0:           # If the right child is left heavy (RL case)
        node.right = self.rotateRight(node.right) # Inner rotation
      node = self.rotateLeft(node)              # Outer rotation (RR or RL after inner)
    return node                                 # Ensure node is always returned

  def __balanceRight(self, node):               # Rebalance after right deletion
    if node.heightDiff() > 1:                   # If node is left heavy
      if node.left.heightDiff() < 0:            # If the left child is right heavy (RR cases)
        node.left = self.rotateLeft(node.left)  # Outer rotation
      node = self.rotateRight(node)             # Inner rotation (RR or RL after outer)
    return node

  # --- Utility ---
  def reset_counters(self):
      self.avl_rotation_count = 0
      self.avl_search_comparison = 0

  # --- HELPER TO SEE THE TREE ---
  def __print_structure(self, node, level):
        if node is not None:
            self.__print_structure(node.right, level + 1)
            print(' ' * 4 * level + f'(L:{level}) ->', node.key)
            self.__print_structure(node.left, level + 1)

# Implementation of Red-Black Tree


In [2]:
# ==========================================
# 2. Red-Black Tree Implementation
# ==========================================

# Color Constants for consistency
RED = 'red'
BLACK = 'black'

class Node:
    """A node in a Red-Black Tree."""
    def __init__(self, key, color=RED):
        self.key = key
        self.color = color
        self.parent = None
        self.left = None
        self.right = None
        # Need data for the assignment structure
        self.data = key, key

class RedBlackTree:
    """Instrumented Red-Black Tree implementation."""
    def __init__(self):
        # Sentinel NIL node (always BLACK)
        self.nil = Node(key=None, color=BLACK)
        self.nil.parent = self.nil
        self.nil.left = self.nil
        self.nil.right = self.nil
        self.root = self.nil

        # --- Introducing Counters ---
        self.rbt_rotation_count = 0
        self.rbt_search_comparison = 0

    # --- Rotation Logic ---

    def left_rotate(self, node_x):
        """Performs a left rotation on node_x, incrementing the rotation counter."""
        self.rbt_rotation_count += 1

        node_y = node_x.right
        node_x.right = node_y.left

        if node_y.left != self.nil:
            node_y.left.parent = node_x

        node_y.parent = node_x.parent

        if node_x.parent == self.nil:
            self.root = node_y
        elif node_x == node_x.parent.left:
            node_x.parent.left = node_y
        else:
            node_x.parent.right = node_y

        node_y.left = node_x
        node_x.parent = node_y

    def right_rotate(self, node_y):
        """Performs a right rotation on node_y, incrementing the rotation counter."""
        self.rbt_rotation_count += 1

        node_x = node_y.left
        node_y.left = node_x.right

        if node_x.right != self.nil:
            node_x.right.parent = node_y

        node_x.parent = node_y.parent

        if node_y.parent == self.nil:
            self.root = node_x
        elif node_y == node_y.parent.right:
            node_y.parent.right = node_x
        else:
            node_y.parent.left = node_x

        node_x.right = node_y
        node_y.parent = node_x

    # --- Insertion Logic (Calls instrumented rotations via fixup) ---

    def rb_insert(self, key):
        """Inserts a key and performs RBT fixup."""
        new_node = Node(key)
        new_node.color = RED # New nodes are always red

        y = self.nil
        x = self.root

        # 1. Standard BST insertion (while loop counts comparisons for traversal)
        while x != self.nil:
            y = x
            if new_node.key < x.key:
                x = x.left
            else:
                x = x.right

        new_node.parent = y
        if y == self.nil:
            self.root = new_node
        elif new_node.key < y.key:
            y.left = new_node
        else:
            y.right = new_node

        new_node.left = self.nil
        new_node.right = self.nil

        self.rb_insert_fixup(new_node)

    def rb_insert_fixup(self, z):
        """Restores RBT properties after insertion."""
        # This logic uses the instrumented rotation methods
        while z.parent.color == RED:
            if z.parent == z.parent.parent.left:
                y = z.parent.parent.right # Uncle
                if y.color == RED:
                    z.parent.color = BLACK
                    y.color = BLACK
                    z.parent.parent.color = RED
                    z = z.parent.parent
                else:
                    if z == z.parent.right:
                        z = z.parent
                        self.left_rotate(z) # Count Rotation
                    z.parent.color = BLACK
                    z.parent.parent.color = RED
                    self.right_rotate(z.parent.parent) # Count Rotation
            else:
                y = z.parent.parent.left # Uncle
                if y.color == RED:
                    z.parent.color = BLACK
                    y.color = BLACK
                    z.parent.parent.color = RED
                    z = z.parent.parent
                else:
                    if z == z.parent.left:
                        z = z.parent
                        self.right_rotate(z) # Count Rotation
                    z.parent.color = BLACK
                    z.parent.parent.color = RED
                    self.left_rotate(z.parent.parent) # Count Rotation
        self.root.color = BLACK

    # --- Search Logic (Instrumented for Comparisons) ---

    def rb_search(self, key):
      """Searches for a key, counts comparisons (2 per node visit), and returns True/False."""
      current_node = self.root
      while current_node != self.nil:

          # --- 2 Node Search Comparison ---
          # Comparison: Check for key match
          self.rbt_search_comparison += 1
          if key == current_node.key:
              # Key found
              return True

          # Comparison: Decide direction (less than/greater than)
          self.rbt_search_comparison += 1
          if key < current_node.key:
              current_node = current_node.left
          else:
              current_node = current_node.right
          # --- End 2 Node Search Comparison ---

          # # --- 1 Node Search Comparison ---
          # self.rbt_search_comparison += 1

          # if key == current_node.key:
          #     # Key found
          #     return True
          # elif key < current_node.key:
          #     current_node = current_node.left
          # else:
          #     current_node = current_node.right
          # # --- End 1 Node Search Comparison ---


      # Key not found
      return False

    # --- Utility Functions (Provided by Haris) ---

    def rb_transplant(self, u, v):
        if u.parent == self.nil:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def tree_minimum(self, node):
        """Finds the minimum node in a subtree, counting comparisons."""
        while node.left != self.nil:
            # COMPARISON COUNT: Check if we've reached the minimum (left is nil)
            self.rbt_search_comparison += 1
            node = node.left
        return node

    # --- Deletion Logic ---

    def rb_delete(self, key):
        z = self.nil
        current_node = self.root


        while current_node != self.nil:
            # # Phase 1: Search for node z (Instrumented for 1 comparison per node)
            # self.rbt_search_comparison += 1 # 🔑 COMPARISON 1 (Search check: Found/Left/Right)

            # if key == current_node.key:
            #     z = current_node
            #     break
            # elif key < current_node.key:
            #     current_node = current_node.left
            # else:
            #     current_node = current_node.right

            # Phase 1: Search for node z (Instrumented for 2 comparison per node)
            self.rbt_search_comparison += 1
            if key == current_node.key:
                z = current_node
                break

            self.rbt_search_comparison += 1
            if key < current_node.key:
                current_node = current_node.left
            else:
                current_node = current_node.right

        if z == self.nil:
            # print(f"Key {key} not found in the tree.") # Suppress printing in experiment
            return

        y = z
        y_original_color = y.color

        if z.left == self.nil:
            x = z.right
            self.rb_transplant(z, z.right)
        elif z.right == self.nil:
            x = z.left
            self.rb_transplant(z, z.left)
        else:
            # Phase 2: Find minimum/successor (Comparisons counted in tree_minimum)
            y = self.tree_minimum(z.right)
            y_original_color = y.color
            x = y.right
            if y.parent == z:
                # This is a pointer assignment, not a key comparison
                x.parent = y
            else:
                self.rb_transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            self.rb_transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color

        if y_original_color == BLACK:
            self.rb_delete_fixup(x)

    def rb_delete_fixup(self, x):
        """Fixes double black violations using recoloring and rotations."""
        while x != self.root and x.color == BLACK:
            if x == x.parent.left:
                w = x.parent.right
                if w.color == RED:
                    w.color = BLACK
                    x.parent.color = RED
                    self.left_rotate(x.parent) # Rotation counted
                    w = x.parent.right
                if w.left.color == BLACK and w.right.color == BLACK:
                    w.color = RED
                    x = x.parent
                else:
                    if w.right.color == BLACK:
                        w.left.color = BLACK
                        w.color = RED
                        self.right_rotate(w) # Rotation counted
                        w = x.parent.right
                    w.color = x.parent.color
                    x.parent.color = BLACK
                    w.right.color = BLACK
                    self.left_rotate(x.parent) # Rotation counted
                    x = self.root
            else: # Symmetric Case
                w = x.parent.left
                if w.color == RED:
                    w.color = BLACK
                    x.parent.color = RED
                    self.right_rotate(x.parent) # Rotation counted
                    w = x.parent.left
                if w.right.color == BLACK and w.left.color == BLACK:
                    w.color = RED
                    x = x.parent
                else:
                    if w.left.color == BLACK:
                        w.right.color = BLACK
                        w.color = RED
                        self.left_rotate(w) # Rotation counted
                        w = x.parent.left
                    w.color = x.parent.color
                    x.parent.color = BLACK
                    w.left.color = BLACK
                    self.right_rotate(x.parent) # Rotation counted
                    x = self.root
        x.color = BLACK


    def inorder_traversal(self, node):
        """Performs an inorder traversal for verification."""
        if node != self.nil:
            self.inorder_traversal(node.left)
            print(f"Key: {node.key}, Color: {node.color}")
            self.inorder_traversal(node.right)

    def reset_counters(self):
        """Resets the performance counters."""
        self.rbt_rotation_count = 0
        self.rbt_search_comparison = 0

    # --- Visualization Method ---
    def print_structure(self):
        """Prints the tree structure sideways in the console."""
        print("\n--- Red-Black Tree Structure ---")
        self._print_recursive(self.root, 0)
        print("--------------------------------")

    def _print_recursive(self, node, level):
        """Recursive helper for printing the tree structure."""
        if node != self.nil:
            # Recursively print the right subtree (top of the console output)
            self._print_recursive(node.right, level + 1)

            # Print the current node with indentation
            indent = '    ' * level
            if node == self.root:
                prefix = "ROOT ->"
            elif node == node.parent.left:
                prefix = "L---->"
            else:
                prefix = "R---->"

            print(f"{indent}{prefix} (L:{level}) {node.key} ({node.color.upper()})")

            # Recursively print the left subtree (bottom of the console output)
            self._print_recursive(node.left, level + 1)



In [3]:
from os import replace
import requests
from collections import Counter
import random
import time

# ==============================================================================
# 3. Main Execution
# ==============================================================================

if __name__ == "__main__":

  # 1. Initilize the trees
  avl = AVLTree()
  rbtree = RedBlackTree()

  # 2. Read the file and prepare the words
  word_list = []

  # 1. Define the URL
  url = "https://raw.githubusercontent.com/mosomo82/COMP_SCI_5501/refs/heads/main/Assignment/Assignment4_Application%20of%20Tree%20Structure%20%26%20Algorithms%20in%20Different%20Fields%20of%20Sciences/raw_data/1000%20Frequent%20Words.txt"

  try:
    print(f"Fecting 1,000 frequent words from Github Link ...")
    reponse = requests.get(url)
    reponse.raise_for_status()                      # Check HTTP link status
    full_text = reponse.text.lower()                # convert input words into lower case
    word_list = full_text.splitlines()
  except requests.exceptions.requests.exceptions.HTTPError as e:
    print(f"Error: Could not fetch data from the URL. Please check the link and your connection.")
    print(f"Details: {e}")
    exit()

  # ---Insertion and Rotation Count
  avl.reset_counters()
  rbtree.reset_counters()
  print("\n" + "="*40)
  print("Testing Insertion and Rotation")
  print("="*40)

  # Store in AVL Tree and check time for insertion
  print(f"Inserting {len(word_list)} words into AVL Tree and Red-Black Tree...")
  avl_start_time = time.time()
  for word in word_list:
    avl.insert(word, word)
  avl_end_time = time.time()
  avl_insert_time = avl_end_time - avl_start_time
  avl_rotation_count = avl.avl_rotation_count
  print("--- Insertion Metrics ---")
  print(f"AVL Tree Rotations: {avl_rotation_count}")
  print(f"AVL Tree Insertion Time: {avl_insert_time:.6f} seconds")

  # Store in Red-Black Tree and check time for insertion
  rbt_start_time = time.time()
  for word in word_list:
    rbtree.rb_insert(word)
  rbt_end_time = time.time()
  rbt_insert_time = rbt_end_time - rbt_start_time
  rbt_rotation_count = rbtree.rbt_rotation_count

  print(f"Red-Black Tree Rotation: {rbt_rotation_count}")
  print(f"Red-Black Tree Insertion Time: {rbt_insert_time:.6f} seconds")


  # print("\nResulting Tree Structure (Sideways):")
  # avl.print_structure()

  # print("\nResulting Tree Structure (Sideways):")
  # rbtree.print_structure()

  # Set up random words from the 1000 word list
  print("\n" + "="*40)
  try:
    n = int(input("Enter the number of words to search for (e.g., 100): "))
    if not (0 < n <= len(word_list)):
        print(f"Invalid input. Please enter a number between 1 and {len(word_list)}. Defaulting to 100.")
        n = 100
  except ValueError:
      print("Invalid input. Please enter an integer. Defaulting to 100.")
      n = 100

  words_to_search = random.sample(word_list, n)

  print(f"Randomly selected {len(words_to_search)} words to search.")
  print(words_to_search)

  # --- Search and Comparison Count ---
  avl.reset_counters()
  rbtree.reset_counters()

  print("\n" + "="*40)
  print("Running Search and Comparison")
  print("="*40)

  # Search in AVL
  avl_start_time = time.time()
  for word in words_to_search:
      avl.search(word)
  avl_end_time = time.time()
  avl_search_time = avl_end_time - avl_start_time
  avl_search_comparison = avl.avl_search_comparison
  print("--- Search Metrics ---")
  print(f"AVL Tree Comparisons: {avl_search_comparison}")
  print(f"AVL Search Time: {avl_search_time:.4f}s")

  # Search in Red-Black Tree
  rbt_search_start_time = time.time()
  for word in words_to_search:
    rbtree.rb_search(word)
  rbt_search_end_time = time.time()
  rbt_search_time = rbt_search_end_time - rbt_start_time
  rbt_search_comparison = rbtree.rbt_search_comparison
  print(f"Red-Black Tree Comparisons: {rbt_search_comparison}")
  print(f"Red-Black Tree Search Time: {rbt_search_time:.4f}s")

  # --- Deletion and Rotation---
  avl.reset_counters()
  rbtree.reset_counters()
  print("\n" + "="*40)
  print("Running Deletion and Rotation")
  print("="*40)

  # Delete same 100 random words for AVL Tree
  print(f"Deleting {len(words_to_search)} words from AVL Tree and Red-Black Tree...")
  avl_del_start_time = time.time()
  for  word in words_to_search:
    avl.delete(word)
  avl_del_end_time = time.time()
  avl_del_time = avl_del_end_time - avl_del_start_time
  avl_deletion_rotation = avl.avl_rotation_count
  avl_deletion_comparison = avl.avl_search_comparison
  print("--- Deletion Metrics ---")
  print(f"AVL Tree Rotation for Deletion: {avl_deletion_rotation}")
  print(f"AVL Tree Comparison for Deletion: {avl_deletion_comparison}")
  print(f"AVL Deletion Time: {avl_del_time:.6f} seconds")

  # Delete same 100 random words for Red-Black Tree
  rbtree_del_start_time = time.time()
  for word in words_to_search:
      rbtree.rb_delete(word)
  rbtree_del_end_time = time.time()
  rbtree_del_time = rbtree_del_end_time - rbtree_del_start_time
  rbtree_deletion_rotation = rbtree.rbt_rotation_count
  rbtree_deletion_comparison = rbtree.rbt_search_comparison
  print(f"Red-Black Tree Rotation for Deletion: {rbtree_deletion_rotation}")
  print(f"Red-Black Tree Comparison for Deletion: {rbtree_deletion_comparison}")
  print(f"Red-Black Tree Deletion Time: {rbtree_del_time:.6f} seconds")

  # --- Final Result Comparison ---
  print("\n" + "="*60)
  print("--- FINAL PERFORMANCE RESULTS ---")
  print("="*60)

  print("\n--- Insertion ---")
  print(f"AVL Tree Rotations:      {avl_rotation_count}")
  print(f"AVL Tree:     {avl_insert_time:.6f} seconds")
  print(f"Red-Black Tree Rotation:       {rbt_rotation_count}")
  print(f"Red-Black Tree Insertion Time:      {rbt_insert_time:.6f} seconds")

  print(f"\n--- Searching (for {len(words_to_search)} random words) ---")
  avg_avl = avl_search_comparison / len(words_to_search)
  avg_rbt = rbt_search_comparison / len(words_to_search)
  print(f"AVL Tree Search Time:     {avl_search_time:.6f} seconds")
  print(f"AVL Tree (Total Comparisons):       {avl_search_comparison} (Avg: {avg_avl:.2f})")
  print(f"Red-Black Tree Search Time:     {rbt_search_time:.6f} seconds")
  print(f"Red-Black Tree (Total Comparisons):       {rbt_search_comparison} (Avg: {avg_rbt:.2f})")

  print(f"\n--- Deletion Time (for {len(words_to_search)} random words) ---")
  print(f"AVL Total Tree Rotation:      {avl_deletion_rotation}")
  print(f"AVL Total Tree Comparison:    {avl_deletion_comparison}")
  print(f"AVL Tree :      {avl_del_time:.6f} seconds")
  print(f"Red-Black Total Tree Rotation:     {rbtree_deletion_rotation}")
  print(f"Red-Black Total Tree Comparison:      {rbtree_deletion_comparison}")
  print(f"Red-Black Tree Deletion Time:     {rbtree_del_time:.6f} seconds")


  # print("\nResulting AVL Tree Structure (Sideways):")
  # avl.print_structure()

  # print("\nResulting Red-Black Tree Structure (Sideways):")
  # rbtree.print_structure()





Fecting 1,000 frequent words from Github Link ...

Testing Insertion and Rotation
Inserting 1000 words into AVL Tree and Red-Black Tree...
--- Insertion Metrics ---
AVL Tree Rotations: 746
AVL Tree Insertion Time: 0.017311 seconds
Red-Black Tree Rotation: 604
Red-Black Tree Insertion Time: 0.003978 seconds

Enter the number of words to search for (e.g., 100): 100
Randomly selected 100 words to search.
['trouble', 'move', 'paint', 'gentle', 'friend', 'then', 'master', 'shop', 'main', 'find', 'still', 'it', 'station', 'hand', 'hundred', 'eye', 'special', 'size', 'the', 'garden', 'top', 'wind', 'early', 'town', 'point', 'take', 'snow', 'present', 'clothe', 'serve', 'no', 'man', 'property', 'north', 'lead', 'stick', 'claim', 'young', 'until', 'quick', 'soft', 'any', 'supply', 'oxygen', 'crease', 'least', 'search', 'sight', 'wheel', 'separate', 'under', 'lie', 'rock', 'machine', 'dollar', 'not', 'again', 'wall', 'bird', 'area', 'skin', 'branch', 'heard', 'equal', 'finger', 'open', 'off', 'b

In [4]:
# # def test_avl_tree():
# #     print("=== TEST 1: Right-Right Case (Should Rotate Left) ===")
# #     # Inserting numbers in ascending order forces the tree to become
# #     # right-heavy immediately. If AVL works, it will balance itself.
# #     tree = AVLTree()
# #     keys = [1, 2, 3, 4, 5, 6, 7]

# #     print(f"Inserting: {keys}")
# #     for k in keys:
# #         tree.insert(k, f"data-{k}")

# #     print("\nResulting Tree Structure (Sideways):")
# #     tree.print_structure()

# #     print("\nAnalysis:")
# #     print("If working: 4 should be the root (or close to it).")
# #     print("If failing: It will look like a ladder (1->2->3...).")

# #     print("\n" + "="*40 + "\n")

# #     print("=== TEST 2: Left-Left Case (Should Rotate Right) ===")
# #     # Inserting in descending order forces left-heaviness.
# #     tree2 = AVLTree()
# #     keys_desc = [10, 9, 8, 7, 6, 5]

# #     print(f"Inserting: {keys_desc}")
# #     for k in keys_desc:
# #         tree2.insert(k, f"data-{k}")

# #     print("\nResulting Tree Structure:")
# #     tree2.print_structure()

# #     print("\n" + "="*40 + "\n")

# #     print("=== TEST 3: Zig-Zag (Double Rotation) ===")
# #     # This requires a Left rotation on child, then Right on parent
# #     tree3 = AVLTree()
# #     # Insert 20 (Root), then 10 (Left). Tree is fine.
# #     # Insert 15. 15 is > 10 but < 20. This is the complex case.
# #     complex_keys = [20, 10, 15]

# #     print(f"Inserting: {complex_keys}")
# #     for k in complex_keys:
# #         tree3.insert(k, f"data-{k}")

# #     print("\nResulting Tree Structure:")
# #     tree3.print_structure()
# #     print("Expectation: 15 should be the new root.")

# def test_avl_deletion():
#     tree = AVLTree()

#     print("=== SETUP: Building a Balanced Tree ===")
#     keys = [40, 20, 60, 10, 30, 50, 70]
#     for k in keys:
#         tree.insert(k, f"data-{k}")

#     tree.print_structure()
#     print("\n" + "="*40 + "\n")

#     print("=== TEST 1: Delete a Leaf (No Rotation needed) ===")
#     print("Deleting 10...")
#     tree.delete(10)
#     tree.print_structure()

#     print("\n" + "="*40 + "\n")

#     print("=== TEST 2: Delete causing Rotation (The Real Test) ===")

#     print("Deleting 50...")
#     tree.delete(50)
#     print("Deleting 70...")
#     tree.delete(70)

#     print("\nResulting Tree after massive deletion:")
#     tree.print_structure()

#     print("\nVERIFICATION:")
#     print("If the tree is a straight line (linked list), the rebalancing FAILED.")
#     print("If the tree looks like a pyramid (short height), the rebalancing WORKED.")

# if __name__ == "__main__":
#     test_avl_deletion()

# if __name__ == "__main__":
#     test_avl_tree()